# Catalog AI: AI-Powered Product Data Improvements

This notebook implements a toy version of the **Catalog AI** system discussed in [Addressing Gen AI's Quality Control Problem](https://hbr.org/2025/09/addressing-gen-ais-quality-control-problem) (Harvard Business Review)

![description](_static/article.png)

Generative AI systems can produce vast numbers of candidate actions, but at scale the central problem is not generation—it is quality control and learning. When Amazon began automating the creation of product pages using large language models, most generated outputs were initially unreliable, and only a small fraction improved business outcomes. The solution was not better prompts or more human review, but a system: automated guardrails to filter low-quality outputs, experimentation to measure impact, and feedback loops that used experimental results to improve future generation. By combining causal measurement, decision rules, and software infrastructure, the system could test millions of hypotheses, discard most of them, and continuously learn from the few that worked. This notebook introduces a simplified, toy version of that idea: how to build a Learn · Decide · Repeat loop that turns large-scale analysis into action and learning over time. 

![description](_static/online-retail-simulator.png)

We use th using the [Online Retail Simulator](https://github.com/eisenhauerIO/tools-catalog-generator). We demonstrate two key capabilities: 1. **LLM-generated product details** - Using local models via Ollama to create unique titles, descriptions, and features
2. **Causal impact measurement** - Simulating and visualizing the effect of improved product content on sales

In [1]:
!pip install -qq git+https://github.com/eisenhauerIO/tools-catalog-generator.git

Before running this notebook, execute the setup script to install and configure Ollama:

```bash
bash setup_catalog_ai.sh
```

The script installs the Online Retail Simulator package and sets up Ollama with the `llama3.2` model.

In [2]:
from online_retail_simulator import simulate, load_job_results
from support import print_product_details

The **Ollama backend** uses a local LLM to generate unique, contextually-aware product titles, descriptions, and features. Custom prompts let you control the tone and style—enabling different positioning for the same products. Let's compare **luxury** vs **budget** positioning for the same products.

In [3]:
# Generate base products
job_info = simulate("config_simulation.yaml")
products = load_job_results(job_info)["products"]

# Select 2 for demonstration
sample_products = products[["asin", "category", "price"]].head(2)
sample_products

,asin,category,price
0,BRPOIG8F1C,Clothing,19.63
1,BO6B9M80O2,Electronics,185.86


In [8]:
!cat prompt_luxury.txt

You are a JSON generator for luxury e-commerce products.
Generate premium, aspirational product details.

Products:
{products_json}

For each product, add:
- title: Elegant product title (50-100 chars) with premium feel
- description: Luxurious description (100-300 chars) emphasizing quality and exclusivity
- brand: Premium-sounding brand name
- features: List of 3-5 premium features

Response must be ONLY valid JSON starting with [ and ending with ].


In [9]:
!cat prompt_budget.txt

You are a JSON generator for budget-friendly e-commerce products.
Generate practical, value-focused product details.

Products:
{products_json}

For each product, add:
- title: Clear product title (50-100 chars) emphasizing value
- description: Practical description (100-300 chars) focusing on functionality and savings
- brand: Approachable brand name
- features: List of 3-5 practical features

Response must be ONLY valid JSON starting with [ and ending with ].


In [5]:
from IPython.display import Code
import inspect
from online_retail_simulator.simulate.product_details_ollama import (
    simulate_product_details_ollama,
)

Code(inspect.getsource(simulate_product_details_ollama), language="python")

def simulate_product_details_ollama(
    products_df: pd.DataFrame,
    model: str = DEFAULT_MODEL,
    ollama_url: str = OLLAMA_URL,
    prompt_path: str = None,
) -> pd.DataFrame:
    """Generate product details using Ollama (local LLM).

    Args:
        products_df: Input products with asin, category, price
        model: Ollama model to use (default: llama3.2)
        ollama_url: Ollama API URL (default: http://localhost:11434)
        prompt_path: Optional path to custom prompt template file.
            Template should contain {products_json} placeholder.

    Returns:
        DataFrame with added title, description, brand, features
    """
    # Load custom prompt or use default
    prompt_template = _load_prompt_template(prompt_path) if prompt_path else PROMPT_TEMPLATE

    results = []
    products = products_df.to_dict("records")

    for i in range(0, len(products), DEFAULT_BATCH_SIZE):
        batch = products[i : i + DEFAULT_BATCH_SIZE]
        prompt = prompt_template.format(products_json=json.dumps(batch, indent=2))

        response = requests.post(
            f"{ollama_url}/api/generate",
            json={
                "model": model,
                "prompt": prompt,
                "stream": False,
            },
            timeout=120,
        )
        response.raise_for_status()

        response_text = response.json().get("response", "")
        batch_results = json.loads(response_text)
        results.extend(batch_results)

    return pd.DataFrame(results)

In [6]:
luxury_products = simulate_product_details_ollama(
    sample_products, prompt_path="prompt_luxury.txt"
)
print_product_details(luxury_products, "Luxury")


LUXURY

  Title: Elegant Silk Blend Blouse with Hand Embroidery
  Brand: Aurum Couture
  Description: Experience the art of luxury with our exquisite silk blend blouse, adorned with intricate hand embroidery that adds a touch of sophistication to any outfit.

  Title: Exquisite 4K Smartwatch with Premium Materials
  Brand: Prestige Watches
  Description: Elevate your style and stay connected in style with our exceptional 4K smartwatch, crafted from premium materials that exude luxury and sophistication.


In [7]:
budget_products = simulate_product_details_ollama(
    sample_products, prompt_path="prompt_budget.txt"
)
print_product_details(budget_products, "Budget")


BUDGET

  Title: Affordable Fashion - Women's Blouse Set (3-Piece)
  Brand: Primark
  Description: Stay stylish and comfortable with this versatile blouse set, perfect for work or everyday wear.

  Title: Budget-Friendly Power Bank - 20,000mAh Portable Charger
  Brand: Aukey
  Description: Stay charged on-the-go with this affordable and reliable portable power bank.


The same product generates completely different content depending on the prompt—enabling A/B testing of content strategies, audience targeting, and channel customization.